# Initial setup

This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

## Install

In [1]:
!pip install datasets
import datasets

## Import

In [2]:
import torch
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

pd.set_option('display.max_columns', None)

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tokenize-ucberkeley-using-distilbert/__results__.html
/kaggle/input/tokenize-ucberkeley-using-distilbert/validation.pkl
/kaggle/input/tokenize-ucberkeley-using-distilbert/train.pkl
/kaggle/input/tokenize-ucberkeley-using-distilbert/test.pkl
/kaggle/input/tokenize-ucberkeley-using-distilbert/validation.csv
/kaggle/input/tokenize-ucberkeley-using-distilbert/__notebook__.ipynb
/kaggle/input/tokenize-ucberkeley-using-distilbert/__output__.json
/kaggle/input/tokenize-ucberkeley-using-distilbert/train.csv
/kaggle/input/tokenize-ucberkeley-using-distilbert/test.csv
/kaggle/input/tokenize-ucberkeley-using-distilbert/custom.css


## Config

In [4]:
from dataclasses import dataclass

@dataclass
class Config:
    model_name = 'distilbert-base-uncased'
    dataset_name = 'ucberkeley-dlab/measuring-hate-speech'
    tokenizer_root = '/kaggle/input/tokenize-ucberkeley-using-distilbert/'
    text_column = 'text'
    # if the raw id column is string, replace that with an integer index during preprocessing
    id_column = 'comment_id'

    # target in raw dataset. However, it will be renamed to `labels` here to facilitate training setup
    raw_target_column = 'hatespeech'
    target_column = 'labels'
    
    # If needs to be splitted into train test validation set
    need_to_split = True
    # if need_to_split is True, test and validation data with each be 50% of this amount
    test_size = 0.3
    max_seq_length = 128
    seed = 2022
    
    
    batch_size = 64
    learning_rate = 1e-3
    epochs = 10
    num_labels = 2

## Set seed

In [5]:
import random

def seed_torch(seed=7):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

global_seed = Config.seed
seed_torch(global_seed)

## Get device

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Load tokenized data

In [7]:
text = Config.text_column
target = Config.target_column

In [8]:
import pickle
    
with open(os.path.join(Config.tokenizer_root, 'train.pkl'), 'rb') as input_file:
    train_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(os.path.join(Config.tokenizer_root, 'validation.pkl'), 'rb') as input_file:
    validation_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(os.path.join(Config.tokenizer_root, 'test.pkl'), 'rb') as input_file:
    test_tokenized = pickle.load(input_file)
    input_file.close()

In [9]:
print(train_tokenized)

Dataset({
    features: ['comment_id', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 26994
})


# Training

## Data loader

In [10]:
BATCH_SIZE = Config.batch_size
id_column = Config.id_column

train_dataloader = DataLoader(train_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE*5)
test_dataloader = DataLoader(test_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE*5)

## Model, hyper-parameters and callbacks

In [11]:
from train_utils import TrainUtil, EarlyStopping, ModelCheckPoint

num_labels = Config.num_labels
model_name = Config.model_name
train_util = TrainUtil(Config.id_column, Config.target_column, device)

In [12]:
model = TrainUtil.load_pretrained_model(model_name, num_labels)

# Define optimizer
LEARNING_RATE = Config.learning_rate
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
EPOCHS = Config.epochs

# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Total parameters count: 66955010
Trainable parameters count: 7680002, percent 11.470


In [13]:
result_dir = ''
best_model_path = result_dir + 'model.pt'
if result_dir != '':
    os.makedirs(result_dir, exist_ok=True)

check_point = ModelCheckPoint(filepath=best_model_path)
early_stopping = EarlyStopping(patience=3, min_delta=0)

## Loop

In [14]:
start_epoch = 1
# load a previous model if there is any
# model, optimizer, lr_scheduler, start_epoch = load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)
model = model.to(device)

for epoch in range(start_epoch, EPOCHS+1):
    gc.collect()
    
    train_loss, train_result, train_probs = train_util.train(model, train_dataloader, optimizer, epoch)
    val_loss, val_result, val_probs = train_util.evaluate(model, validation_dataloader, epoch, 'Validation')

    print(
      f"Epoch: {epoch} | "
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Validation loss: {val_loss:.3f} | "
      f"Validation result: {val_result} | "
    )
    
    loss = -val_result['f1']
    lr_scheduler.step(loss)
    check_point(model, optimizer, lr_scheduler, epoch, loss)
    
    early_stopping(loss)
    if early_stopping.early_stop:
        break
    print()

Epoch 1 (Train):   0%|          | 0/422 [00:00<?, ?it/s]

Epoch 1 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch: 1 | Train loss: 0.450 | Train result: {'accuracy': 0.7887, 'f1': 0.5662, 'auc': 0.824} |
Validation loss: 0.438 | Validation result: {'accuracy': 0.7896, 'f1': 0.4876, 'auc': 0.8467} | 

Loss improved from inf to -0.488. Saving model.



Epoch 2 (Train):   0%|          | 0/422 [00:00<?, ?it/s]

Epoch 2 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

...

Epoch 9 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch: 9 | Train loss: 0.385 | Train result: {'accuracy': 0.8228, 'f1': 0.6601, 'auc': 0.8763} |
Validation loss: 0.443 | Validation result: {'accuracy': 0.8024, 'f1': 0.6161, 'auc': 0.8459} | 

Loss improved from -0.587 to -0.616. Saving model.



Epoch 10 (Train):   0%|          | 0/422 [00:00<?, ?it/s]

Epoch 10 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch: 10 | Train loss: 0.383 | Train result: {'accuracy': 0.8234, 'f1': 0.6607, 'auc': 0.8776} |
Validation loss: 0.444 | Validation result: {'accuracy': 0.8017, 'f1': 0.6126, 'auc': 0.8445} | 
Early stopping counter 1 of 3



In [15]:
# load the best model
model, _, _, best_epoch = TrainUtil.load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)

train_loss, train_result, train_probs = train_util.evaluate(model, train_dataloader, best_epoch, 'Train')
# no need to reevaluate if the validation set if the last model is the best one
if best_epoch != epoch:
    val_loss, val_result, val_probs = train_util.evaluate(model, validation_dataloader, best_epoch, 'Validation')
test_loss, test_result, test_probs = train_util.evaluate(model, test_dataloader, best_epoch, 'Test')

Loaded best model from epoch 9


Epoch 9 (Train):   0%|          | 0/422 [00:00<?, ?it/s]

Epoch 9 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 9 (Test):   0%|          | 0/19 [00:00<?, ?it/s]

## Dump results

In [16]:
# Save the results
train_util.dump_results(
    result_dir,train_probs, train_tokenized, 
    val_probs, validation_tokenized, test_probs, test_tokenized
)

## Dump config

In [17]:
import json

config_dict = dict(Config.__dict__)
# exclude hidden variables
keys = list(config_dict.keys())
for key in keys:
    if key.startswith('__'):
        del config_dict[key]
        
with open(os.path.join(result_dir, 'config.json'), 'w') as output:
    json.dump(config_dict, output, indent=4)